# LMA: Levenberg-Marquardt Algorithm Operator

The `LMA` operator implements the Levenberg-Marquardt algorithm, an iterative procedure widely used for solving non-linear least squares problems or for finding roots of non-linear systems of equations. This implementation is designed to be robust, but at the same time provides sensible defaults to facilitate its usage.

## Introduction

The LMA method interpolates between the more aggressive Gauss-Newton algorithm and the more conservative method of gradient descent.

It is an iterative algorithm. Every iteration, the residual and the Jacobian are calculated for a given set of parameters (the "current guess"). Then a new guess $p-\Delta p$ is calculated, such that:

$$(J^T J + \lambda D)\Delta p = J^T y$$

where $J$ is the Jacobian ($J_{ij}=\partial y_i / \partial p_j$), $D$ is a diagonal matrix such that $D_{kk} = \max(\epsilon, (J^T J)_{kk})$ with $\epsilon$ an arbitrarily small number, $y$ is the residual, and $\lambda$ is the damping factor. The damping factor determines how much the next guess approximates the prediction of the Gauss-Newton algorithm (lower damping factors) or the gradient descent methods (higher damping factors), in effect defining a trust-region.

For this new guess, the predicted error reduction is calculated as:

$$\Delta s_p = \frac{1}{2}(\Delta p^T J^T y + \lambda \Delta p^T D \Delta p)$$

If the ratio of the actual error reduction calculated during the next iteration with respect to this prediction is above a defined limit, the current guess is accepted and the damping factor decreased. Else, the new guess is rejected and the damping factor is increased.

The algorithm finishes once one of these conditions is met, returning the last accepted guess:

* Maximum numbers of iterations reached
* Residual below specified tolerance
* Relative change in parameters or residual below specified tolerance
* Maximum damping factor reached

### Normalized damping factor

A particularity of this LMA implementation is the use of a *normalized damping factor*. The damping factor $\lambda$ will oscillate between the specified limits $\lambda_{min}$ and $\lambda_{max}$, starting with an initial value $\lambda_0$. The normalized damping factor is calculated as:

$$\lambda_{norm} = \frac{(\lambda_{max}-\lambda_0)(\lambda-\lambda_{min})}{(\lambda_0-\lambda_{min})(\lambda_{max}-\lambda)}$$

If $\lambda_{norm}$ is 1, the initial damping factor is being used. If it is larger, it means that more damping than indicated was necessary, reaching the maximum $\lambda_{max}$ at infinite; if it is lower, it means that it could be decreased for faster convergence, with the point at 0 corresponding with $\lambda_{min}$.

The usage of normalized damping factors allows to monitor and adjust the effective size of the trust region during successive function calls independently of the actual damping parameters at use.

## `LMA` Operator

    R←{X}f LMA Y

`f` is a configuration namespace or a function. Configuration namespaces can define the following configuration parameters (else, the provided defaults are used):

* `toli`: Maximum number of iterations (default `1E6`)
* `tols`: Tolerance for the sum of squared residuals (default `tolr`)
* `tolr`: Tolerance for relative change, either in the solution or the residual (default `⎕CT`)
* `tolg`: Tolerance for the gain ratio to accept or reject a step (default `1E¯2`)
* `dini`: Initial damping factor for `d=1` (default `1E¯2`)
* `dinc`: Increment of damping factor after rejected solution (default `5`)
* `ddec`: Decrement of damping factor after accepted solution (default `÷dinc`)
* `dmax`: Maximum damping factor (default `÷⎕CT`)
* `dmin`: Minimum damping factor (default `÷dmax`)

With the exception of `toli` and `tols`, these parameters should be modified only by expert users.

Additionally, the namespace must contain two functions:

* `Callback`: Callback function (default `⊢`)
* `Eval`: Evaluation function

The evaluation function must return the residual and Jacobian for the set of parameters given as right argument, and an optional left argument `X`.
In case `f` is a function, the default configuration is used, setting the `Eval` function to `f`.
The callback function is called every iteration before checking convergence, with a solution namespace as argument
and discarding the return value.

A solution namespace is a configuration namespace with the additional elements:

* `iter`: Number of iterations
* `sse`: Sum of squared residuals
* `rel`: Relative change metric
* `p0`: Initial guess
* `p`: Current guess

`Y` is a vector. If the depth of the vector is not larger than 1, it is enclosed first.
The first element of `Y` (or `⊂Y` if `1=≡⍵`) contains the initial guess for the parameters.
Additional elements of `Y` must be either configuration namespaces (overwriting the parameters at right with those at left) or a number indicating the value of the initial normalized damping factor.

The returned value `R` is a solution namespace.


## `LM` Operator

`LM` is a simplified version of `LMA`. Usage:

    R←X f LM g Y

where `f` is the evaluation function, `g` is the callback function (which takes as argument an `iter sse rel dnorm p` vector), `Y` is a two elements vector with the initial normalized damping factor and parameters guess. `X` is a vector defining the parameters `toli tols tolr tolg dini dinc ddec dmax dmin`. The return value `R` will be an `iter sse rel dnorm p` vector.

## Implementation

In [1]:
)clear

clear ws

In [2]:
]dinput
LM←{ti ts tr tg d0 di dd dx dn←⍺ ⋄ d p←⍵               ⍝ Levenberg–Marquardt algorithm

    D←((dx-d0)×dn-⍨⊢)÷(d0-dn)×dx-⊢                     ⍝ damping factor normalization(λ)
    E←⍺⍺{s←+.×⍨⊃y j←⍺⍺ ⍵ ⋄ s ⍵ y j}                    ⍝ eval(p): sum(error²), parameters, residual, jacobian
    A←{s p y j←⍵ ⋄ s p,(t+.×j)(y+.×⍨t←⍉j)}             ⍝ accept(EG output): sum(error²), parameters, JtJ, Jty
    
    T←{                                                ⍝ try guess(λ)
        r⊢←0 ⋄ 11::dx⌊⍵×di ⋄ d⊢←D ⍵                    ⍝     bad guess if domain error
        ∆p←jy⌹jj+⍺×⍤1⊢⍵×dj←⎕CT⌈1 1⍉jj                  ⍝     change of parameters
        s0←2÷⍨∆p+.×jy+∆p×⍵×dj                          ⍝     predicted error decrement
        s1←s-⊃g←E⊢q←p-∆p                               ⍝     actual error decrement
        r⊢←(p(-÷⍥(+.×⍨)⊣)q)⌊|s1÷s                      ⍝     relative change in parameters or residuals
        (⎕CT>s0)∧⎕CT<s1:dx⌊⍵×di                        ⍝     if no changing, increase damping
        (⎕CT≤s0)∧tg≥s1÷s0:dx⌊⍵×di                      ⍝     if diverging, increase damping
        dn⌈⍵×dd⊣s p jj jy⊢←A g                         ⍝     accept change, decrease damping
    }
    C←⍵⍵{                                              ⍝ convergence check(λ_prev, λ)
        _←⍺⍺(i⊢←i+1)s r d p                            ⍝     call user function
        (ti<i)∨(dx∧.=⍺ ⍵)∨(ts>s)∨(r>0)∧tr>r            ⍝     iterations, max damping, residual, not changing
    }
    i r←0 ⋄ _←⍵⍵ i s r d p⊣s p jj jy←A E⊃⌽⍵            ⍝ init
    i s r d p⊣(∘.=⍨⍳≢p)T⍣C{11::dx ⋄ D⍣¯1⊢⍵}d           ⍝ return iterations, sse, change, norm damping, parameters
}

In [3]:
]dinput
LMA←{⍺←⊢ ⋄ p←⊃w←⊆⍵ ⋄ 0::⎕SIGNAL ⎕EN                    ⍝ pass signals

    n←'d',¨'ini' 'inc' 'dec' 'max' 'min'               ⍝ damping
    n←('isrg',⍨¨⊂'tol'),n                              ⍝ tolerances
    nc←'tols' 'ddec' 'dmin'                            ⍝ computed defaults
    nr←'iter' 'sse' 'rel' 'dnorm' 'p'                  ⍝ results
    c←(                                                ⍝ default config
        toli:1e6 ⋄ tolr:⎕CT ⋄ tolg:1e¯2                ⍝     tolerances
        dini:1e¯2 ⋄ dinc:5 ⋄ dmax:÷⎕CT                 ⍝     damping
        p0:p ⋄ verbose:0                               ⍝     initial parameters and logging
    )
    F←{1((⊂6 0⍕↑),12 ¯5∘⍕¨⍤↓)⍵} ⋄ P←{⎕←F ⍵}            ⍝ format and print
    DF←{⍕'sp',¨':',¨(⊂2 5)⌷F ⍵}                        ⍝ display form and set results

    3=⎕NC'⍺⍺':⍺((⍺⍺{⍵.Eval←⍺⍺ ⋄ ⍵}c)∇∇)w               ⍝ ⍺⍺ is Eval function
    (1<≢w)∧~2|⎕DR⊃⌽w:⍺((⎕NS ⍺⍺(⊃⌽w))∇∇)¯1↓w            ⍝ non numeric extra argument is config
    2<≢w:⎕SIGNAL 11                                    ⍝ wrong argument
    
    c.CallBack←⊢ ⋄ c←⎕NS c ⍺⍺ ⋄ c.dnorm←1⊣⍣(1=≢w)⊃⌽w   ⍝ default callback and actual config
    _←c ⎕VSET(↑nc)(⎕VGET(↑nc)c.(tolr,÷dinc dmax))      ⍝ computed defaults
    c←⍺{(3=⎕NC'⍺⍺')∧0≠⍵.⎕NC'Left':⍵ ⋄ ⍵.Left←⍺⍺ ⋄ ⍵}c  ⍝ left arg
    CB←{c.CallBack c⎕VSET(↑nr)⍵}⊣P⍣(c.verbose≢0)       ⍝ callback function
    c⊣c.⎕DF DF(c⎕VGET↑n)c.(Left∘Eval)LM CB c.dnorm p   ⍝ return namespace
}

In [4]:
]link.export -overwrite # .

## Examples

In [5]:
NOISY←{⍵+⍺×0.5-⍨?0⍴⍨≢⍵}

### Exponential decay

In [6]:
FN←{A k C←⍺ ⋄ C+A×*-k×⍵} ⋄ _EV_←{A k C←⍵ ⋄ xe←A×⍺⍺×e←*-k×⍺⍺ ⋄ (⍵⍵-⍵ FN ⍺⍺)(⍉(-e)⍪xe⍪⍉⍪-=⍨e)}
p←10 0.5 1 ⋄ y←0.25 NOISY p FN x←(⍳10)-1 ⋄ ExpDecay←{x y←⍺ ⋄ x _EV_ y⊢⍵} ⋄ ⊂5⍕↑x y(p FN x)(⊃x y ExpDecay p)
1e6 ⎕CT ⎕CT 1e¯2 1e¯2 5(÷5)(÷⎕CT)⎕CT(x y∘ExpDecay)LM⊢1(5 0.1 0.5) ⍝ ti ts tr tg d0 di dd dx dn
⊢s←x y ExpDecay LMA(5 0.1 0.5)(verbose:1)

┌────────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 10.91764 6.99509 4.61949 3.33679 2.30358 1.77714 1.47851 1.33376 1.14604 1.07627│
│ 11.00000 7.06531 4.67879 3.23130 2.35335 1.82085 1.49787 1.30197 1.18316 1.11109│
│ ¯0.08236 ¯0.07022 ¯0.05930 0.10549 ¯0.04977 ¯0.04371 ¯0.01936 0.03178 ¯0.03712 ¯0.03482│
└────────────────────────────────────────────────────────────────────────────────────────┘

┌─┬─────────────┬───────────────┬──────────────┬─────────────────────────────────────┐
│6│0.02174417066│8.497795557E¯16│0.000319999999│9.927861624 0.4955205482 0.9755915254│
└─┴─────────────┴───────────────┴──────────────┴─────────────────────────────────────┘

0 4.9987E1 0.0000E0 1.0000E0 5.0000E0 1.0000E¯1 5.0000E¯1 
 1 2.3674E1 2.5942E¯1 1.0000E0 6.5178E0 3.7706E¯1 2.5427E0 
 2 2.2937E¯1 2.4951E¯1 2.0000E¯1 9.7223E0 5.4644E¯1 1.1462E0 
 3 2.6436E¯2 6.5197E¯4 4.0000E¯2 9.9110E0 4.9302E¯1 9.9055E¯1 
 4 2.1744E¯2 5.1652E¯6 8.0000E¯3 9.9278E0 4.9551E¯1 9.7558E¯1 
 5 2.1744E¯2 2.8373E¯11 1.6000E¯3 9.9279E0 4.9552E¯1 9.7559E¯1 
 6 2.1744E¯2 8.4978E¯16 3.2000E¯4 9.9279E0 4.9552E¯1 9.7559E¯1 
 s: 2.1744E¯2 p: 9.9279E0 4.9552E¯1 9.7559E¯1

In [7]:
⊢ed←x y ExpDecay LMA⊂5 0.1 0.5
ed LMA ed.p0

s: 2.1744E¯2 p: 9.9279E0 4.9552E¯1 9.7559E¯1

s: 2.1744E¯2 p: 9.9279E0 4.9552E¯1 9.7559E¯1

### Linear system

In [8]:
FN←{a b←⍺ ⋄ a+b×⍵} ⋄ _EV_←{a b←⍵ ⋄ (⍵⍵-⍵ FN ⍺⍺)(⍉↑-(=⍨⍺⍺)⍺⍺)}
p←0.1 2 ⋄ y←1e¯3 NOISY p FN x←(⍳10)-1
Linear←x _EV_ y ⋄ ⊂5⍕↑x y(p FN x)(⊃Linear p)
Linear LMA(0 0)(dinc:1e3 ⋄ verbose:1)

┌────────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 0.09982 2.09961 4.10034 6.10013 8.09951 10.10008 12.09979 14.10037 16.09996 18.09973│
│ 0.10000 2.10000 4.10000 6.10000 8.10000 10.10000 12.10000 14.10000 16.10000 18.10000│
│ ¯0.00018 ¯0.00039 0.00034 0.00013 ¯0.00049 0.00008 ¯0.00021 0.00037 ¯0.00004 ¯0.00027│
└────────────────────────────────────────────────────────────────────────────────────────┘

0 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 1 1.1581E3 0.0000E0 1.0000E0 0.0000E0 0.0000E0 
 2 1.1581E3 0.0000E0 1.0000E3 0.0000E0 0.0000E0 
 3 1.1581E3 0.0000E0 1.0000E6 0.0000E0 0.0000E0 
 4 1.1581E3 0.0000E0 1.0000E9 0.0000E0 0.0000E0 
 5 1.1581E3 3.4300E¯10 1.0001E12 9.0999E¯10 2.0158E¯10 
 6 1.1581E3 3.4300E¯7 1.0000E9 9.1090E¯7 2.0178E¯7 
 7 1.1577E3 3.4291E¯4 1.0000E6 9.1072E¯4 2.0175E¯4 
 8 8.4794E2 2.6756E¯1 1.0000E3 7.5750E¯1 1.7256E¯1 
 9 3.2732E¯1 9.9961E¯1 1.0000E0 3.8699E¯1 1.9370E0 
 10 7.6470E¯7 2.2137E¯2 1.0000E¯3 9.9919E¯2 2.0000E0 
 11 7.6346E¯7 1.0033E¯10 1.0000E¯6 9.9899E¯2 2.0000E0 
 12 7.6346E¯7 4.1135E¯25 9.9900E¯10 9.9899E¯2 2.0000E0 
 s: 7.6346E¯7 p: 9.9899E¯2 2.0000E0

### Gauss function

In [9]:
X←{m s←⍺ ⋄ *-(×⍨⍵-m)÷(2××⍨s)} ⋄ FN←{a s m c←⍺ ⋄ c+a×m s X ⍵}
_EV_←{a s m c←⍵ ⋄ xe←m s X ⍺⍺ ⋄ x1←xe×a×(⍺⍺-m)÷×⍨s ⋄ x2←x1×(⍺⍺-m)÷s ⋄ (⍵⍵-⍵ FN ⍺⍺)(⍉↑(-xe)x1(-x2)(-=⍨xe))}
p←10 5 1.5 2 ⋄ y←0 NOISY p FN x←(⍳10)-1 ⋄ Gaussian←{x y←⍺ ⋄ (x _EV_ y)⍵} ⋄ ⊂5⍕↑x y(p FN x)(⊃x y Gaussian p)
(x y Gaussian LMA(8 4 1 1)(dini:1e¯3 ⋄ tols:1e¯6)).(iter sse rel dnorm p)
(x y Gaussian LMA(8 4 1 1)1e¯1(tols:1e¯6)).(iter sse rel dnorm p)
(x y Gaussian LMA(8 4.5 1.2 1.5)(tolr:1e¯3)).(iter sse rel dnorm p)

┌─────────────────────────────────────────────────────────────────────────────────────┐
│ 0.00000 1.00000 2.00000 3.00000 4.00000 5.00000 6.00000 7.00000 8.00000 9.00000│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 11.55997 11.95012 11.95012 11.55997 10.82497 9.82705 8.66977 7.46074 6.29557 5.24652│
│ 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000 0.00000│
└─────────────────────────────────────────────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬──────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│1220703140│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴──────────┴───────────────────────────────────────────────┘

┌──┬───────────┬───────────────┬─────────┬───────────────────────────────────────────────┐
│18│3.779645742│1.377060143E¯15│122070314│8.921655369 3.871347834 2.623807742 3.046908918│
└──┴───────────┴───────────────┴─────────┴───────────────────────────────────────────────┘

┌─┬────────────┬───────────────┬──┬───────────────────────────────────────────────┐
│7│0.9170106519│0.0007211535235│25│9.471886955 4.249297788 2.168664717 2.526269831│
└─┴────────────┴───────────────┴──┴───────────────────────────────────────────────┘